In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os

In [2]:
dir_path = "./demos/blastSpring/blast_spring/"

In [3]:
def getDemoPaths(dir_path):
    demoPaths = []
    x = 0
    for filename in os.listdir(dir_path):
        f = os.path.join(dir_path, filename)
        for demo in os.listdir(f):
            demoPaths.append(os.path.join(f, demo))
    return demoPaths

In [4]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [5]:
def generateData(parsers):
    mapData, killData, nadeData, flashData, roundData, damageData = [], [], [], [], [], []
    for x in parsers:
        x.parse()
        mapData.append(x.parse(return_type="df"))
        killData.append(x._parse_kills(return_type="df"))
        nadeData.append(x._parse_grenades(return_type="df"))
        flashData.append(x._parse_flashes(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (mapData, killData, nadeData, flashData, roundData, damageData)

In [6]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [7]:
def calculateVolumeKillsByPlaces(players, kills, side):
    places = []
    temp = kills.copy()
    for x in temp.MapName.unique():
        temp2 = temp[temp['MapName'] == x]
        for y in temp2.VictimAreaName.unique():
            places.append(temp2.MapName.unique()[0] + ":" + y)
    ret = {}
    for y in players:
        temp = {}
        for x in places:
            maps = x.split(':')[0]
            place = x.split(':')[1]
            k = len(kills[kills['MapName'] == maps][kills['AttackerName'] == y][kills['AttackerAreaName'] == place][kills['AttackerSide'] == side].index)
            temp[x] = k 
        ret[y] = temp
    return ret

In [8]:
def calculateKDDifferentialByPlaces(players, kills, side):
    places = []
    temp = kills.copy()
    for x in temp.MapName.unique():
        temp2 = temp[temp['MapName'] == x]
        for y in temp2.VictimAreaName.unique():
            places.append(temp2.MapName.unique()[0] + ":" + y)
    ret = {}
    for y in players:
        temp = {}
        for x in places:
            maps = x.split(':')[0]
            place = x.split(':')[1]
            k = len(kills[kills['MapName'] == maps][kills['AttackerName'] == y][kills['AttackerAreaName'] == place][kills['AttackerSide'] == side].index)
            d = len(kills[kills['MapName'] == maps][kills['VictimName'] == y][kills['VictimAreaName'] == place][kills['VictimSide'] == side].index)
            #kills = 
            temp[x] = k - d
        ret[y] = temp
    return ret

In [9]:
def printMagnitudeCounts(kdDiff):
    magnitudecounts = {}
    for index, row in kdDiff.iterrows():
        count = 0
        for y in row.keys():
            count += abs(row[y])
        magnitudecounts[index] = count
    for w in sorted(magnitudecounts, key=magnitudecounts.get, reverse=True):
        print(w, magnitudecounts[w])

In [10]:
def convertKDToArrayWithDict(kd):
    mapDf = []
    for i, row in kd.iterrows():
        for z in row.keys():
            mapDf.append({"map" : i.split(":")[0], "Spot" : i.split(":")[1], "Player" : z, "K/D" : row[z]})
    return mapDf

In [11]:
def calculateVolumeHEDamageSide(players, damage, side):
    counts = {}
    for x in players:
        counts[x] = damage[damage['AttackerName'] == x][damage['Weapon'] == 'HE Grenade'][damage['AttackerSide'] == side].HpDamageTaken.sum()
    return counts

In [12]:
def calculateVolumeHEDamage(players, damage):
    counts = {}
    for x in players:
        counts[x] = damage[damage['AttackerName'] == x][damage['Weapon'] == 'HE Grenade'].HpDamageTaken.sum()
    return counts

In [13]:
def calculateTotalKills(players, kills):
    counts = {}
    for x in players:
        counts[x] = len(kills[kills['AttackerName'] == x].index)
    return counts

In [14]:
def calculateMapsPlayed(players_of_interest, kills):
    counts = {}
    for x in players_of_interest:
        for y in kills:
            if x not in counts.keys():
                counts[x] = 0
            if(x in y.AttackerName.value_counts()):
                counts[x] += 1
    return counts

In [15]:
def calculateRoundsPlayed(players, rounds, kills):
    counts = {}
    for x in players:
        for y in range(len(rounds)):
            if(not kills[y][kills[y]['AttackerName'] == x].empty):
                if(kills[y][kills[y]['AttackerName'] == x].AttackerTeam.iloc[0] in rounds[y].WinningTeam.value_counts() or kills[y][kills[y]['AttackerName'] == x].AttackerTeam.iloc[y] in rounds[y].LosingTeam.value_counts()):
                    if(x not in counts.keys()):
                        counts[x] = rounds[y].RoundNum.iloc[-1]
                    else:
                        counts[x] += rounds[y].RoundNum.iloc[-1]
    return counts

In [16]:
def calculateside_volume_flashbangs(players, nades, side):
    counts = {}
    for x in players:
         counts[x] = len(nades[nades['PlayerSide'] == side][nades['GrenadeType'] == 'Flashbang'][nades['PlayerName'] == x].index)
    return counts

In [17]:
demoPaths = getDemoPaths(dir_path)

In [20]:
type(demoPaths[24])

str

In [ ]:
demoParsers = generateDemoParsers(demoPaths)

In [ ]:
blastMapData, blastKillData, blastNadeData, blastFlashData, blastRoundData, blastDamageData = generateData(demoParsers)

In [ ]:
failedDemos = []
blastMapsData, blastKillsData, blastNadesData, blastFlashsData, blastRoundsData, blastDamagesData = [], [], [], [], [], []
for x in range(len(blastKillData)):
    if(blastKillData[x].shape[0] > 100):
        blastMapsData.append(blastMapData[x])
        blastKillsData.append(blastKillData[x])
        blastNadesData.append(blastNadeData[x])
        blastFlashsData.append(blastFlashData[x])
        blastRoundsData.append(blastRoundData[x])
        blastDamagesData.append(blastDamageData[x])
    else:
        failedDemos.append(x)

In [ ]:
for x in failedDemos:
    print(str(x) + " : "  + demoPaths[x])

In [ ]:
demoPaths[26]

In [ ]:
blastRoundData[36]

In [ ]:
len(blastRoundsData)

In [ ]:
len(blastKillsData)

In [ ]:
killData = concat_data(blastKillsData)
nadeData = concat_data(blastNadesData)
flashData = concat_data(blastFlashsData)
roundData = concat_data(blastRoundsData)
damageData = concat_data(blastDamagesData)

In [ ]:
players_of_interest = []
for x in killData.AttackerName.value_counts().index:
    players_of_interest.append(x)

In [ ]:
mapsPlayed = calculateMapsPlayed(players_of_interest, blastKillData)
mapsPlayed

In [ ]:
roundsPlayed = calculateRoundsPlayed(players_of_interest, blastRoundsData, blastKillsData)
roundsPlayed

In [ ]:
fpr, fpr30 = {}, {}
totalKills = calculateTotalKills(players_of_interest, killData)
for x in roundsPlayed.keys():
    fpr[x] = totalKills[x] / roundsPlayed[x]
    fpr30[x] = fpr[x] * 30

In [ ]:
fpr

In [ ]:
fpr30

In [ ]:
teamVolumeFprs = {}
for x in fpr30.keys():
    if killData[killData['AttackerName'] == x].AttackerTeam.iloc[0] not in teamVolumeFprs.keys():
        teamVolumeFprs[killData[killData['AttackerName'] == x].AttackerTeam.iloc[0]] = fpr30[x]
    else:
        teamVolumeFprs[killData[killData['AttackerName'] == x].AttackerTeam.iloc[0]] += fpr30[x]
teamVolumeFprs

In [ ]:
killers = killData.groupby(["AttackerName"]).size().reset_index(name="Kills")
deaths = killData.groupby(["VictimName"]).size().reset_index(name="Deaths")
kdr = killers.merge(deaths, left_on = "AttackerName", right_on = "VictimName")
kdr["KDR"] = kdr["Kills"]/kdr["Deaths"]
kdr = kdr[["AttackerName", "Kills", "Deaths", "KDR"]]
kdr.columns = ["PlayerName", "Kills", "Deaths", "KDR"]
kdr.sort_values(by=["KDR"], ascending=False).head(25)

In [ ]:
killers = killData.groupby(["AttackerName"]).size().reset_index(name="Kills")
deaths = killData.groupby(["VictimName"]).size().reset_index(name="Deaths")
kdr = killers.merge(deaths, left_on = "AttackerName", right_on = "VictimName")
kdr["KDR"] = kdr["Kills"]/kdr["Deaths"]
kdr = kdr[["AttackerName", "Kills", "Deaths", "KDR"]]
kdr.columns = ["PlayerName", "Kills", "Deaths", "KDR"]
kdr.sort_values(by=["KDR"], ascending=True).head(25)

In [ ]:
blastMapData[0]['Damages'].columns

In [ ]:
killData.groupby(["AttackerName", "AttackerTeam"]).IsHeadshot.mean().reset_index(name="HeadShotPct").sort_values("HeadShotPct", ascending=False).head(15)

In [ ]:
blastRoundsData[0]

In [ ]:
killData.groupby(["AttackerName", "AttackerTeam"]).IsHeadshot.mean().reset_index(name="HeadShotPct").sort_values("HeadShotPct", ascending=True).head(15)

In [ ]:
headshots = killData.groupby(["AttackerName", "Weapon"]).IsHeadshot.agg(["mean", "count"]).reset_index()
headshots = headshots[headshots["count"] > 3]
headshots[headshots['mean'] != 1.0].sort_values("mean", ascending=False).head(30)

In [ ]:
headshots.sort_values("mean", ascending=False).head(30)

# Utility
## Flashes

In [ ]:
nadeData.columns

In [ ]:
for i, r in nadeData.groupby(['GrenadeType']).PlayerName.value_counts()['Flashbang'].iteritems():
    print(i + " Threw " + str(r) + " Flashes over " + str(roundsPlayed[i]) + " Rounds  for a " + str(r / roundsPlayed[i]) + " FPR")

In [ ]:
flashData.AttackerTeam.value_counts()

## Nades

In [ ]:
nade_dmg = damageData[damageData["Weapon"].isin(["Incendiary Grenade", "Molotov", "HE Grenade"])]
nade_dmg = damageData.replace("Incendiary Grenade", "Incendiary")
nade_dmg = damageData.replace("Molotov", "Incendiary")
nade_dmg_df = nade_dmg.groupby("AttackerName").HpDamageTaken.sum().reset_index(name="UtilityDamage").sort_values("UtilityDamage", ascending=False)
nade_dmg_df.columns = ["PlayerName", "UtilityDamage"]

In [ ]:
nade_dmgs = []
roundsPlayedArray = []
teams = []
for i, r in nade_dmg_df.iterrows():
    nade_dmgs.append(r['UtilityDamage'] / roundsPlayed[r['PlayerName']])
    roundsPlayedArray.append(roundsPlayed[r['PlayerName']])
    teams.append(damageData[damageData['AttackerName'] == r['PlayerName']].AttackerTeam.iloc[0])

In [ ]:
nade_dmg_df['Util Damage Per Round'] = nade_dmgs
nade_dmg_df['Rounds played'] = roundsPlayedArray
nade_dmg_df['Team'] = teams

In [ ]:
nade_dmg_df.groupby("Team")['Util Damage Per Round'].sum()

In [ ]:
nade_dmg_df

In [ ]:
nade_dmg_df.sort_values(by=['Util Damage Per Round'], ascending=False).head(30)

In [ ]:
nade_dmg_df.sort_values(by=['Util Damage Per Round'], ascending=False).tail(30)

## Smokes

In [ ]:
nadeData.head(50)

In [ ]:
mirageSmokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade'][nadeData['MapName'] == 'de_mirage']
infernoSmokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade'][nadeData['MapName'] == 'de_inferno']
dustSmokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade'][nadeData['MapName'] == 'de_dust2']
vertigoSmokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade'][nadeData['MapName'] == 'de_vertigo']
trainSmokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade'][nadeData['MapName'] == 'de_train']
overpassSmokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade'][nadeData['MapName'] == 'de_overpass']
nukeSmokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade'][nadeData['MapName'] == 'de_nuke']

In [ ]:
mirageCT = mirageSmokes[mirageSmokes['PlayerSide'] == 'CT']
mirageT = mirageSmokes[mirageSmokes['PlayerSide'] == 'T']
infernoCT = infernoSmokes[infernoSmokes['PlayerSide'] == 'CT']
infernoT = infernoSmokes[infernoSmokes['PlayerSide'] == 'T']
dustCT = dustSmokes[dustSmokes['PlayerSide'] == 'CT']
dustT = dustSmokes[dustSmokes['PlayerSide'] == 'T']
vertigoCT = vertigoSmokes[vertigoSmokes['PlayerSide'] == 'CT']
vertigoT = vertigoSmokes[vertigoSmokes['PlayerSide'] == 'T']
trainCT = trainSmokes[trainSmokes['PlayerSide'] == 'CT']
trainT = trainSmokes[trainSmokes['PlayerSide'] == 'T']
overpassCT = overpassSmokes[overpassSmokes['PlayerSide'] == 'CT']
overpassT = overpassSmokes[overpassSmokes['PlayerSide'] == 'T']
nukeCT = nukeSmokes[nukeSmokes['PlayerSide'] == 'CT']
nukeT = nukeSmokes[nukeSmokes['PlayerSide'] == 'T']

In [ ]:
ax = mirageCT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Mirage CT Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Mirage CT Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = mirageT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Mirage T Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Mirage T Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = infernoCT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Inferno CT Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Inferno CT Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = infernoT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Inferno T Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Inferno T Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = dustCT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Dust2 CT Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Dust2 CT Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = dustT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Dust2 T Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Dust2 T Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = vertigoCT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Vertigo CT Smoke Distribution');
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = vertigoT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Vertigo T Smoke Distribution');
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = trainCT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Train CT Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Train CT Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = trainT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Train T Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Train T Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = overpassCT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Overpass CT Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Overpass CT Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = overpassT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Overpass T Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Overpass T Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = nukeCT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Nuke CT Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Nuke CT Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
ax = nukeT.GrenadeAreaName.value_counts(normalize=True).nlargest(10).plot(kind='bar', title='Blast Spring Nuke T Smoke Distribution');
vals = ax.get_yticks()
ax.tick_params(axis='x', colors='red')
ax.tick_params(axis='y', colors='red')
ax.set_title(label ='Blast Spring Nuke T Smoke Distribution', fontdict={'color' : 'red'})
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals]);

In [ ]:
smokes = nadeData[nadeData['GrenadeType'] == 'Smoke Grenade']
smokes

In [ ]:
mapsPlayed